In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [69]:
# 加载和准备数据
data = pd.read_csv('earthquake.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.time
data['Timestamp'] = data.apply(lambda row: pd.Timestamp(f"{row['Date']} {row['Time']}"), axis=1)
data['Timestamp'] = data['Timestamp'].view('int64') // 10**9 #Unix时间戳
features = data[['Timestamp', 'Longitude', 'Latitude']]
targets = data[['Depth', 'Magnitude']]

In [70]:
X_np=features.values
y_np=targets.values
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_x.fit(X_np)
scaler_y.fit(y_np)

MinMaxScaler()

In [71]:
# 使用相同的参数对训练数据和测试数据进行归一化
X_normalized = scaler_x.transform(X_np)
y_normalized = scaler_y.transform(y_np)

In [72]:
def create_sliding_windows(X, Y, window_size):
    X_windows, Y_windows = [], []
    for i in range(len(X) - window_size):
        # 从X中提取窗口
        X_window = X[i:i+window_size]
        # 从Y中提取紧接着窗口的下一个值作为标签
        Y_window = Y[i+window_size]
        X_windows.append(X_window)
        Y_windows.append(Y_window)
    return np.array(X_windows), np.array(Y_windows)

window_size = 10
X_windows, Y_windows = create_sliding_windows(X_normalized, y_normalized, window_size)

X = torch.tensor(X_windows, dtype=torch.float32)
Y = torch.tensor(Y_windows, dtype=torch.float32)

# 确定测试集的大小
test_size = 0.2

# 计算测试集应有的数据点数量
num_data_points = len(X)
num_test_points = int(num_data_points * test_size)

# 计算测试集和训练集的分割点
split_point = num_data_points - num_test_points

# 按顺序划分数据为训练集和测试集
X_train = X[:split_point]
X_test = X[split_point:]
y_train = Y[:split_point]
y_test = Y[split_point:]
# 创建DataLoader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
batch_size=64
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [73]:
device = torch.device("cpu")

In [74]:
# 定义神经网络结构
class ComplexNet(nn.Module):
    def __init__(self):
        super(ComplexNet, self).__init__()
        self.fc1 = nn.Linear(3, 64)  # 输入层到第一个隐藏层
        self.fc2 = nn.Linear(64, 128)  # 第一个隐藏层到第二个隐藏层
        self.fc3 = nn.Linear(128, 64)  # 第二个隐藏层到第三个隐藏层
        self.fc4 = nn.Linear(64, 2)  # 第三个隐藏层到输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# 初始化网络
model_BP = ComplexNet()
model_BP.to(device)

class ComplexLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_prob):
        super(ComplexLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # 定义LSTM层
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_prob)
        
        # 添加Dropout层
        self.dropout = nn.Dropout(dropout_prob)
        
        # 定义额外的全连接层
        self.fc1 = nn.Linear(hidden_dim, hidden_dim * 2)
        self.bn1 = nn.BatchNorm1d(hidden_dim * 2)  # 批量归一化层
        self.fc2 = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, x):
        # 初始化隐藏状态和细胞状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        # 前向传播LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # 只需要LSTM最后一层的输出
        out = out[:, -1, :]
        out = self.dropout(out)
        
        # 通过全连接层
        out = self.fc1(out)
        out = self.bn1(out)
        out = torch.relu(out)
        out = self.fc2(out)
        return out

# 初始化模型
input_dim = 3 # 维度、经度、时间
hidden_dim = 64
num_layers = 2
output_dim = 2 # 震深和震级
dropout_prob = 0.5

model_lstm = ComplexLSTMModel(input_dim, hidden_dim, output_dim, num_layers, dropout_prob)

# 将模型移到GPU
model_lstm = model_lstm.to(device)

# 定义位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# 定义Transformer模型
class TransformerModel(nn.Module):
    def __init__(self, input_dim, seq_len, hidden_dim, output_dim, num_layers, dropout_prob):
        super(TransformerModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.pos_encoder = PositionalEncoding(hidden_dim, dropout_prob)

        # 由于3是质数，我们可以选择将输入特征投影到更高的维度
        self.embedding = nn.Linear(input_dim, hidden_dim)

        encoder_layers = TransformerEncoderLayer(d_model=hidden_dim, nhead=8, 
                                                 dim_feedforward=hidden_dim, dropout=dropout_prob)
        self.transformer_encoder = TransformerEncoder(encoder_layer=encoder_layers, num_layers=num_layers)

        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.embedding(x)  # 将输入特征投影到更高的维度
        x = self.dropout(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # Transformer expects (Seq Len, Batch, Features)
        out = self.transformer_encoder(x)
        out = self.fc_out(out[-1, :, :])  # 取最后一个时间步
        return out

# 初始化模型参数
input_dim = 3  # 时间、经度、纬度
seq_len = 20   # 序列长度
hidden_dim = 64  # 隐藏层的维度
num_layers = 2  # Transformer层的数量
output_dim = 2  # 震深和震级
dropout_prob = 0.5  # Dropout概率

# 创建模型
model_transformer = TransformerModel(input_dim, seq_len, hidden_dim, output_dim, num_layers, dropout_prob)

# 将模型移到GPU
model_transformer = model_transformer.to(device)

In [75]:
# 加载最佳模型
model_BP.load_state_dict(torch.load('lastloss_BP_model.pth'))
model_BP.to(device)  # 确保模型在正确的设备上

ComplexNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
)

In [76]:
# 加载最佳模型
model_lstm.load_state_dict(torch.load('best_lstm_model.pth'))
model_lstm.to(device)  # 确保模型在正确的设备上

ComplexLSTMModel(
  (lstm): LSTM(3, 64, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [77]:
# 加载最佳模型
model_transformer.load_state_dict(torch.load('best_transformer_model.pth'))
model_transformer.to(device)  # 确保模型在正确的设备上

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (embedding): Linear(in_features=3, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=64, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.5, inplace=False)
        (dropout2): Dropout(p=0.5, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_feat

In [78]:
def BP_forward(model, time, Latitude, Longitude):
    '''
    time:1965-01-02 13:44:18
    
    '''
    # 切换到评估模式
    model.eval()
    time = pd.to_datetime(time)
    time=time.timestamp() #Unix时间戳
    features=np.array([[Latitude, Longitude, time]])
    X = torch.tensor(features, dtype=torch.float32)
    # 不计算梯度
    with torch.no_grad():
        X = X.to(device)
        output = model(X)
    return output


In [79]:
BP_forward(model_BP, '1983-01-02 13:44:18', 19.246, 145.616)

tensor([[ 1.3994, -1.1599]])

In [80]:
def lstm_transformer_forward(model, X):
    model.eval() # 将模型设置为评估模式
    predictions = []
    with torch.no_grad(): # 不计算梯度

        X = X.to(device)
        outputs = model(X)
        predictions.append(outputs.cpu().numpy())
        
    # 反标准化以比较实际值
    
    predictions = np.vstack(predictions)
    predictions = scaler_y.inverse_transform(predictions)
    return predictions

In [81]:
for inputs, targets in test_loader:
    X=inputs[1]
    X=X.reshape(1,10,3)
    break

In [82]:
lstm_transformer_forward(model_lstm, X)

array([[24.418081 ,  5.4246664]], dtype=float32)

In [83]:
lstm_transformer_forward(model_transformer, X)

array([[99.51853  ,  5.9539104]], dtype=float32)